- Секция "сабмит" - это полное решение от загрузки данных до формирования сабмита на хакатоне

- Секции "без сабмита" - это экспериментальная работа с train, без test, без сабмитов. Там идеи и реализация

- Осталное можно не смотреть

1. Baseline
   - Время/календарь: weekday, hour_of_day, hour_week, hod_sin, hod_cos, wkd_sin, wkd_cos
   - CTR (time-aware, бета-сглаживание, без утечек): C15, C16, C17, C18, C19, C20, C21 - это  фичи вида Cxx_ctr
   - Частотность появлений: cumcount для C15–C21 → фичи Cxx_cumcount
   - Базовые категориальные (в модели): banner_pos, device_type, device_conn_type, C1

2. SOTA
   - Время/календарь: weekday, hour_of_day, hour_week, hod_sin, hod_cos, wkd_sin, wkd_cos
   - CTR (time-aware, бета-сглаживание, без утечек): C15–C21, site_category, app_category, banner_pos, C1, device_type, device_conn_type → *_ctr
   - OOF-CTR (для низко/средне-кардинальных): site_category, app_category, banner_pos, C1 → *_ctr_oof
   - Частотность появления:
     1. cumcount для C15–C21
     2. дополнительные cumcount для: site_id, site_domain, app_id, app_domain, device_model
     3. logcount = log1p(cumcount) для SOLO-колонок это *_logcount
   - Базовые категориальные (в модели): banner_pos, device_type, device_conn_type, C1, weekday, hour_of_day (кодировались как каткоды)

   - Примечание: recency и кросс-признаки сознательно не добавлялись (ограничения памяти и анти-утечки)

## смотрим на данные

In [ ]:
train

,idx,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,1,2,15706,320,50,1722,0,35,-1,79
1,2.0,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,1,0,15704,320,50,1722,0,35,100084,79
2,3.0,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,1,0,15704,320,50,1722,0,35,100084,79
3,5.0,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,...,1,0,18993,320,50,2161,0,35,-1,157
4,7.0,1.000072e+19,0,14102100,1005,0,8fda644b,25d4cfcd,f028772b,ecad2386,...,1,0,20362,320,50,2333,0,39,-1,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40388930,NaN,9.998753e+18,1,14103023,1005,1,e151e245,7e091613,f028772b,ecad2386,...,1,0,17262,320,50,1872,3,39,100173,23
40388931,NaN,9.999038e+18,0,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,9c13b419,...,1,2,23160,320,50,2667,0,47,-1,221
40388932,NaN,9.999585e+18,0,14103023,1005,1,f61eaaae,6b59f079,f028772b,ecad2386,...,1,0,20969,320,50,2372,0,813,-1,46
40388933,NaN,9.999636e+18,1,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,3c4b944d,...,1,0,16859,320,50,1887,3,39,100194,23


In [9]:
train.shape

(40388935, 25)

In [10]:
train.columns

Index(['idx', 'id', 'click', 'hour', 'C1', 'banner_pos', 'site_id',
       'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category',
       'device_id', 'device_ip', 'device_model', 'device_type',
       'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21'],
      dtype='object')

In [11]:
train.dtypes

idx                 float64
id                  float64
click                 int64
hour                  int64
C1                    int64
banner_pos            int64
site_id              object
site_domain          object
site_category        object
app_id               object
app_domain           object
app_category         object
device_id            object
device_ip            object
device_model         object
device_type           int64
device_conn_type      int64
C14                   int64
C15                   int64
C16                   int64
C17                   int64
C18                   int64
C19                   int64
C20                   int64
C21                   int64
dtype: object

In [ ]:
train.isna().sum()

idx                 40228967
id                         0
click                      0
hour                       0
C1                         0
banner_pos                 0
site_id                    0
site_domain                0
site_category              0
app_id                     0
app_domain                 0
app_category               0
device_id                  0
device_ip                  0
device_model               0
device_type                0
device_conn_type           0
C14                        0
C15                        0
C16                        0
C17                        0
C18                        0
C19                        0
C20                        0
C21                        0
dtype: int64

In [13]:
train["click"].value_counts()

click
0    33530927
1     6858008
Name: count, dtype: int64

In [ ]:
train.nunique()

idx                   159968
id                  40388935
click                      2
hour                     240
C1                         7
banner_pos                 7
site_id                 4736
site_domain             7742
site_category             26
app_id                  8550
app_domain               559
app_category              36
device_id            2684975
device_ip            6725249
device_model            8249
device_type                5
device_conn_type           4
C14                     2626
C15                        8
C16                        9
C17                      435
C18                        4
C19                       68
C20                      172
C21                       60
dtype: int64

In [ ]:
train.duplicated().sum()

0

In [4]:
train["hour_dt"] = pd.to_datetime(train["hour"].astype(str), format="%y%m%d%H", utc=True)
test["hour_dt"]  = pd.to_datetime(test["hour"].astype(str),  format="%y%m%d%H", utc=True)

train["weekday"]     = train["hour_dt"].dt.weekday.astype("int8")
train["hour_of_day"] = train["hour_dt"].dt.hour.astype("int8")
test["weekday"]      = test["hour_dt"].dt.weekday.astype("int8")
test["hour_of_day"]  = test["hour_dt"].dt.hour.astype("int8")

In [21]:
for c in ["hour_of_day", "weekday"]:
    if c in train.columns:
        g = train.groupby(c)["click"].mean().sort_index()
        rho = pd.Series(g.index).corr(pd.Series(g.values), method="spearman")
        print(f"Spearman({c}, CTR) = {rho:.3f}")


Spearman(hour_of_day, CTR) = -0.230
Spearman(weekday, CTR) = 0.571


In [5]:
train

,idx,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,...,C15,C16,C17,C18,C19,C20,C21,hour_dt,weekday,hour_of_day
0,1.0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,320,50,1722,0,35,-1,79,2014-10-21 00:00:00+00:00,1,0
1,2.0,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,320,50,1722,0,35,100084,79,2014-10-21 00:00:00+00:00,1,0
2,3.0,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,320,50,1722,0,35,100084,79,2014-10-21 00:00:00+00:00,1,0
3,5.0,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,...,320,50,2161,0,35,-1,157,2014-10-21 00:00:00+00:00,1,0
4,7.0,1.000072e+19,0,14102100,1005,0,8fda644b,25d4cfcd,f028772b,ecad2386,...,320,50,2333,0,39,-1,157,2014-10-21 00:00:00+00:00,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40388930,NaN,9.998753e+18,1,14103023,1005,1,e151e245,7e091613,f028772b,ecad2386,...,320,50,1872,3,39,100173,23,2014-10-30 23:00:00+00:00,3,23
40388931,NaN,9.999038e+18,0,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,9c13b419,...,320,50,2667,0,47,-1,221,2014-10-30 23:00:00+00:00,3,23
40388932,NaN,9.999585e+18,0,14103023,1005,1,f61eaaae,6b59f079,f028772b,ecad2386,...,320,50,2372,0,813,-1,46,2014-10-30 23:00:00+00:00,3,23
40388933,NaN,9.999636e+18,1,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,3c4b944d,...,320,50,1887,3,39,100194,23,2014-10-30 23:00:00+00:00,3,23


In [6]:
m = 5
cols_ctr = ["C15","C16","C17","C18","C19","C20","C21"]

In [7]:
last_day = train["hour_dt"].dt.date.max()

mask_past  = train["hour_dt"].dt.date < last_day
mask_valid = train["hour_dt"].dt.date == last_day

In [ ]:
p_global = train.loc[mask_past, "click"].mean()
p_global

0.1698596491451188

In [ ]:
for c in cols_ctr:
    new_col = f"{c}_ctr"
    train[new_col] = np.nan

train

,idx,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,...,hour_dt,weekday,hour_of_day,C15_ctr,C16_ctr,C17_ctr,C18_ctr,C19_ctr,C20_ctr,C21_ctr
0,1.0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,2014-10-21 00:00:00+00:00,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,2014-10-21 00:00:00+00:00,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,2014-10-21 00:00:00+00:00,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,...,2014-10-21 00:00:00+00:00,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7.0,1.000072e+19,0,14102100,1005,0,8fda644b,25d4cfcd,f028772b,ecad2386,...,2014-10-21 00:00:00+00:00,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40388930,NaN,9.998753e+18,1,14103023,1005,1,e151e245,7e091613,f028772b,ecad2386,...,2014-10-30 23:00:00+00:00,3,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40388931,NaN,9.999038e+18,0,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,9c13b419,...,2014-10-30 23:00:00+00:00,3,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40388932,NaN,9.999585e+18,0,14103023,1005,1,f61eaaae,6b59f079,f028772b,ecad2386,...,2014-10-30 23:00:00+00:00,3,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40388933,NaN,9.999636e+18,1,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,3c4b944d,...,2014-10-30 23:00:00+00:00,3,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
for c in cols_ctr:
    past = train.loc[mask_past, [c, "click"]]
    shows  = past.groupby(c)["click"].count()
    clicks = past.groupby(c)["click"].sum()

    ctr_raw = clicks / shows
    ctr_raw.loc[shows < m] = p_global

    new_col = f"{c}_ctr"

    train.loc[mask_valid, new_col] = train.loc[mask_valid, c].map(ctr_raw.to_dict()).fillna(p_global)


In [11]:
train

,idx,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,...,hour_dt,weekday,hour_of_day,C15_ctr,C16_ctr,C17_ctr,C18_ctr,C19_ctr,C20_ctr,C21_ctr
0,1.0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,2014-10-21 00:00:00+00:00,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,2014-10-21 00:00:00+00:00,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,2014-10-21 00:00:00+00:00,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,...,2014-10-21 00:00:00+00:00,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7.0,1.000072e+19,0,14102100,1005,0,8fda644b,25d4cfcd,f028772b,ecad2386,...,2014-10-21 00:00:00+00:00,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40388930,NaN,9.998753e+18,1,14103023,1005,1,e151e245,7e091613,f028772b,ecad2386,...,2014-10-30 23:00:00+00:00,3,23,0.158597,0.158331,0.302971,0.148226,0.243136,0.267409,0.211582
40388931,NaN,9.999038e+18,0,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,9c13b419,...,2014-10-30 23:00:00+00:00,3,23,0.158597,0.158331,0.105378,0.159072,0.138729,0.193124,0.164029
40388932,NaN,9.999585e+18,0,14103023,1005,1,f61eaaae,6b59f079,f028772b,ecad2386,...,2014-10-30 23:00:00+00:00,3,23,0.158597,0.158331,0.114030,0.159072,0.114030,0.193124,0.097772
40388933,NaN,9.999636e+18,1,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,3c4b944d,...,2014-10-30 23:00:00+00:00,3,23,0.158597,0.158331,0.147488,0.148226,0.243136,0.082037,0.211582


In [12]:
# быстрый контроль: сколько заполнилось в валид-день
for c in cols_ctr:
    nc = f"{c}_ctr"
    print(nc, "filled:", train.loc[mask_valid, nc].notna().mean())

C15_ctr filled: 1.0
C16_ctr filled: 1.0
C17_ctr filled: 1.0
C18_ctr filled: 1.0
C19_ctr filled: 1.0
C20_ctr filled: 1.0
C21_ctr filled: 1.0


In [13]:
train.loc[mask_valid, ["hour_dt","click"] + cols_ctr + [f"{c}_ctr" for c in cols_ctr]].head()

,hour_dt,click,C15,C16,C17,C18,C19,C20,C21,C15_ctr,C16_ctr,C17_ctr,C18_ctr,C19_ctr,C20_ctr,C21_ctr
36169997,2014-10-30 00:00:00+00:00,0,320,50,2695,0,35,100019,51,0.158597,0.158331,0.114623,0.159072,0.166601,0.164256,0.183651
36169998,2014-10-30 00:00:00+00:00,0,320,50,2695,0,35,100020,51,0.158597,0.158331,0.114623,0.159072,0.166601,0.072760,0.183651
36169999,2014-10-30 00:00:00+00:00,0,320,50,1973,3,39,100148,23,0.158597,0.158331,0.181369,0.148226,0.243136,0.234147,0.211582
36170000,2014-10-30 00:00:00+00:00,0,320,50,2227,0,935,-1,48,0.158597,0.158331,0.112823,0.159072,0.119831,0.193124,0.138517
36170001,2014-10-30 00:00:00+00:00,0,320,50,2716,3,47,-1,23,0.158597,0.158331,0.204518,0.148226,0.138729,0.193124,0.211582


## Сабмит: SOTA

In [1]:
import os
import gc
import time
import numpy as np
import pandas as pd
import lightgbm as lgb

pd.options.mode.chained_assignment = None

def optimize_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if pd.api.types.is_integer_dtype(df[col]):
            mn, mx = df[col].min(), df[col].max()
            if mn >= 0:
                if mx < 2**8:    df[col] = df[col].astype("uint8")
                elif mx < 2**16: df[col] = df[col].astype("uint16")
                elif mx < 2**32: df[col] = df[col].astype("uint32")
                else:            df[col] = df[col].astype("uint64")
            else:
                if   -2**7  <= mn < 2**7:   df[col] = df[col].astype("int8")
                elif -2**15 <= mn < 2**15:  df[col] = df[col].astype("int16")
                elif -2**31 <= mn < 2**31:  df[col] = df[col].astype("int32")
                else:                        df[col] = df[col].astype("int64")
    return df

def create_time_features(df: pd.DataFrame) -> pd.DataFrame:
    df["hour_dt"]    = pd.to_datetime(df["hour"].astype(str), format="%y%m%d%H", utc=True)
    df["weekday"]    = df["hour_dt"].dt.weekday.astype("int8")
    df["hour_of_day"]= df["hour_dt"].dt.hour.astype("int8")
    df["hour_week"]  = (df["weekday"].astype("int16")*24 + df["hour_of_day"].astype("int16")).astype("int16")
    df["hod_sin"]    = np.sin(2*np.pi*df["hour_of_day"]/24).astype("float32")
    df["hod_cos"]    = np.cos(2*np.pi*df["hour_of_day"]/24).astype("float32")
    df["wkd_sin"]    = np.sin(2*np.pi*df["weekday"]/7).astype("float32")
    df["wkd_cos"]    = np.cos(2*np.pi*df["weekday"]/7).astype("float32")
    return df

def time_aware_ctr_for_train(train: pd.DataFrame, cols: list, alpha: float=10.0):
    dates = np.sort(train["hour_dt"].dt.date.unique())
    p_gl  = float(train["click"].mean())
    outnames=[]
    for col in cols:
        out=f"{col}_ctr"; train[out]=np.nan; outnames.append(out)
    for i,d in enumerate(dates):
        if i==0: continue
        m_day  = (train["hour_dt"].dt.date == d)
        m_prev = (train["hour_dt"].dt.date <  d)
        prior  = float(train.loc[m_prev, "click"].mean()) if m_prev.any() else p_gl
        for col in cols:
            stats  = train.loc[m_prev, [col,"click"]].groupby(col, observed=True)["click"].agg(["count","sum"])
            smooth = (stats["sum"] + alpha*prior) / (stats["count"] + alpha)
            train.loc[m_day, f"{col}_ctr"] = train.loc[m_day, col].map(smooth.to_dict()).fillna(prior).astype("float32")
    for col in cols:
        train[f"{col}_ctr"] = train[f"{col}_ctr"].fillna(p_gl).astype("float32")
    return outnames

def ctr_map_from_full_train(train: pd.DataFrame, cols: list, alpha: float=10.0):
    p = float(train["click"].mean()); maps={}
    for col in cols:
        agg = train.groupby(col, observed=True)["click"].agg(["count","sum"])
        maps[col] = ((agg["sum"] + alpha*p) / (agg["count"] + alpha)).astype("float32").to_dict()
    return maps, p

def apply_ctr_map(df: pd.DataFrame, maps: dict, default_ctr: float):
    for col, mp in maps.items():
        out=f"{col}_ctr"
        df[out] = df[col].map(mp).fillna(default_ctr).astype("float32")

def add_oof_ctr_train_only(train_df: pd.DataFrame, col: str, dates_sorted, mask_past, mask_valid, alpha=5.0):
    out=f"{col}_ctr_oof"; train_df[out]=np.nan
    p_gl = float(train_df.loc[mask_past,"click"].mean())
    agg  = train_df.loc[mask_past,[col,"click"]].groupby(col)["click"].agg(["count","sum"])
    ctr_valid = (agg["sum"] + alpha*p_gl)/(agg["count"] + alpha)
    train_df.loc[mask_valid, out] = train_df.loc[mask_valid, col].map(ctr_valid.to_dict()).fillna(p_gl)
    for i,d in enumerate(dates_sorted):
        if i==0: continue
        m_day  = (train_df["hour_dt"].dt.date == d)
        m_prev = (train_df["hour_dt"].dt.date <  d)
        p_prev = float(train_df.loc[m_prev,"click"].mean())
        agg_p  = train_df.loc[m_prev,[col,"click"]].groupby(col)["click"].agg(["count","sum"])
        ctr_p  = (agg_p["sum"] + alpha*p_prev)/(agg_p["count"] + alpha)
        train_df.loc[m_day,out] = train_df.loc[m_day,col].map(ctr_p.to_dict()).fillna(p_prev)
    train_df[out] = train_df[out].astype("float32"); return out

def cumcount_train_and_test_map(train_df: pd.DataFrame, test_df: pd.DataFrame, col: str, time_col="hour_dt"):
    tmp = train_df[[col, time_col]].copy()
    tmp["_idx"] = train_df.index
    tmp.sort_values(time_col, inplace=True)
    cc = tmp.groupby(col, sort=False).cumcount().astype("int32")
    out = f"{col}_cumcount"
    train_df[out] = 0
    train_df.loc[tmp["_idx"].values, out] = cc.values
    train_df[out] = train_df[out].astype("float32")
    freq_map = train_df.groupby(col, observed=True).size().astype("int32").to_dict()
    test_df[out] = test_df[col].map(freq_map).fillna(0).astype("float32")
    del tmp, cc, freq_map; gc.collect()
    return out

def encode_cats_threeway(tr_df, va_df, te_df, cols):
    code_cols=[]
    for c in cols:
        tr_cat = tr_df[c].astype("category"); va_cat = va_df[c].astype("category"); te_cat = te_df[c].astype("category")
        cats = tr_cat.cat.categories.union(va_cat.cat.categories).union(te_cat.cat.categories)
        tr_df[f"{c}__code"] = tr_cat.cat.set_categories(cats).cat.codes.astype("int32")
        va_df[f"{c}__code"] = va_cat.cat.set_categories(cats).cat.codes.astype("int32")
        te_df[f"{c}__code"] = te_cat.cat.set_categories(cats).cat.codes.astype("int32")
        code_cols.append(f"{c}__code")
    return code_cols

def main():
    t0 = time.perf_counter()
    train  = pd.read_csv("train.csv")
    test   = pd.read_csv("test.csv")
    sample = pd.read_csv("sample_submission.csv")
    if "idx" not in sample.columns:
        raise ValueError("В sample_submission.csv нет колонки 'idx'.")

    # Время/календарь
    train = create_time_features(train)
    test  = create_time_features(test)

    # Даты/маски
    dates_sorted = np.sort(train["hour_dt"].dt.date.unique())
    if len(dates_sorted) < 2:
        raise ValueError("Нужно >=2 уникальных дат в train.")
    last_day  = dates_sorted[-1]
    first_day = dates_sorted[0]
    mask_valid = (train["hour_dt"].dt.date == last_day)
    mask_train = (train["hour_dt"].dt.date < last_day) & (train["hour_dt"].dt.date != first_day)
    mask_past  = (train["hour_dt"].dt.date < last_day)

    # CTR: time-aware (train) + map (test)
    ctr_cols = [c for c in ["site_category","app_category","banner_pos","C1",
                            "device_type","device_conn_type",
                            "C15","C16","C17","C18","C19","C20","C21"] if c in train.columns]
    ctr_cols_created = time_aware_ctr_for_train(train, ctr_cols, alpha=10.0)
    ctr_maps, default_ctr = ctr_map_from_full_train(train, ctr_cols, alpha=10.0)
    apply_ctr_map(test, ctr_maps, default_ctr)

    # OOF-CTR только для low-card
    low_card_for_ctr = [c for c in ["site_category","app_category","banner_pos","C1"] if c in train.columns]
    ctr_low_cols=[]
    for c in low_card_for_ctr:
        ctr_low_cols.append(add_oof_ctr_train_only(train, c, dates_sorted, mask_past, mask_valid, alpha=5.0))
    maps_low, def_low = ctr_map_from_full_train(train, low_card_for_ctr, alpha=5.0)
    for col, mp in maps_low.items():
        out = f"{col}_ctr_oof"
        test[out] = test[col].map(mp).fillna(def_low).astype("float32")

    # Cumcount ТОЛЬКО для C15–C21: train честный, test = train_count map
    cumcount_cols=[]
    for c in [x for x in ["C15","C16","C17","C18","C19","C20","C21"] if x in train.columns]:
        cumcount_cols.append(cumcount_train_and_test_map(train, test, c))

    # Признаки
    base_cats = ["banner_pos","device_type","device_conn_type","C1","weekday","hour_of_day"]
    base_cats = [c for c in base_cats if c in train.columns and c in test.columns]
    time_num_cols = ["hour_week","hod_sin","hod_cos","wkd_sin","wkd_cos"]
    time_num_cols = [c for c in time_num_cols if c in train.columns]

    num_cols = time_num_cols + cumcount_cols + ctr_low_cols + ctr_cols_created

    cols_for_tr = base_cats + num_cols + ["click"]
    cols_for_te = [c for c in base_cats + num_cols if c in test.columns]

    mask_drop_first = (train["hour_dt"].dt.date == first_day)
    train_df = train.loc[mask_train & ~mask_drop_first, cols_for_tr].copy()
    valid_df = train.loc[mask_valid,                          cols_for_tr].copy()
    test_df  = test[cols_for_te].copy()

    train_df = optimize_dtypes(train_df); valid_df = optimize_dtypes(valid_df); test_df = optimize_dtypes(test_df)
    for df in (train_df, valid_df, test_df):
        for c in num_cols:
            if c in df.columns:
                df[c] = df[c].astype("float32", copy=False)

    cat_code_cols = encode_cats_threeway(train_df, valid_df, test_df, base_cats)

    X_tr = pd.concat([train_df[cat_code_cols], train_df[num_cols]], axis=1)
    X_va = pd.concat([valid_df[cat_code_cols], valid_df[num_cols]], axis=1)
    X_te = pd.concat([test_df[cat_code_cols],  test_df[num_cols]],  axis=1)
    y_tr = train_df["click"].astype(np.uint8).values
    y_va = valid_df["click"].astype(np.uint8).values

    cat_idx = list(range(len(cat_code_cols)))

    del train, test, train_df, valid_df, test_df
    gc.collect()

    lgb_params = dict(
        objective="binary", metric="auc",
        learning_rate=0.01,
        num_leaves=256,             
        max_depth=-1,
        min_data_in_leaf=50,        
        lambda_l2=2.0,              
        feature_fraction=0.75,
        bagging_fraction=0.8, bagging_freq=1,
        max_bin=255,
        device_type="cuda", gpu_device_id=0,
        seed=42, verbosity=-1, num_threads=-1
    )

    dtrain = lgb.Dataset(X_tr, label=y_tr, categorical_feature=cat_idx, free_raw_data=True)
    dvalid = lgb.Dataset(X_va, label=y_va, categorical_feature=cat_idx, reference=dtrain, free_raw_data=True)

    print("===> Обучение (валидация последним днём)...")
    cb = [lgb.early_stopping(stopping_rounds=75, first_metric_only=True),
          lgb.log_evaluation(period=100)]
    bst = lgb.train(lgb_params, dtrain, num_boost_round=700,
                    valid_sets=[dvalid], valid_names=["valid"], callbacks=cb)
    best_iter = bst.best_iteration or bst.current_iteration()
    print(f"Best iteration: {best_iter}")

    # Финальное обучение (OOM-safe, GPU→CPU fallback)
    print("===> Финальное обучение на всём train (train+valid)...")
    try:
        X_full = pd.concat([X_tr, X_va], axis=0, ignore_index=True)
        y_full = np.concatenate([y_tr, y_va])
        del dtrain, dvalid, X_tr, X_va, y_tr, y_va, bst
        gc.collect()
        dfull = lgb.Dataset(X_full, label=y_full, categorical_feature=cat_idx, free_raw_data=True)
        try:
            bst_full = lgb.train(lgb_params, dfull, num_boost_round=best_iter)
        except Exception as e:
            print("GPU OOM, fallback CPU:", repr(e))
            lgb_params_cpu = {**lgb_params, "device_type": "cpu"}
            bst_full = lgb.train(lgb_params_cpu, dfull, num_boost_round=best_iter)
        del dfull, X_full, y_full
        gc.collect()
    except Exception as e:
        print("Не удалось собрать full train, используем модель с валидации:", repr(e))
        bst_full = bst

    print("===> Предсказания...")
    test_pred = bst_full.predict(X_te, num_iteration=best_iter).astype("float32")

    test_idx = pd.read_csv("test.csv", usecols=["idx"]) if os.path.exists("test.csv") else None
    if test_idx is not None and len(test_idx)==len(test_pred):
        pred_df = pd.DataFrame({"idx": test_idx["idx"].values, "click": test_pred})
        submit = sample[["idx"]].merge(pred_df, on="idx", how="left")
    else:
        submit = pd.DataFrame({"idx": sample["idx"].values, "click": test_pred})

    if submit["click"].isna().any():
        submit["click"] = submit["click"].fillna(float(submit["click"].mean()))

    submit.to_csv("submission.csv", index=False)
    print("Сохранено: submission.csv")
    print(f"Готово за {time.perf_counter()-t0:.1f} c")

if __name__ == "__main__":
    main()


===> Обучение (валидация последним днём)...
Training until validation scores don't improve for 150 rounds
[100]	valid's auc: 0.693136
[200]	valid's auc: 0.694732
[300]	valid's auc: 0.696154
[400]	valid's auc: 0.698204
[500]	valid's auc: 0.700364
[600]	valid's auc: 0.701686
[700]	valid's auc: 0.701683
[800]	valid's auc: 0.701823
[900]	valid's auc: 0.702071
[1000]	valid's auc: 0.701895
Early stopping, best iteration is:
[885]	valid's auc: 0.702134
Evaluated only: auc
Best iteration: 885
===> Финальное обучение на всём train (train+valid)...
===> Предсказания...
Сохранено: submission.csv
Готово за 1509.4 c


## Сабмит: Baseline

In [2]:
import lightgbm as lgb
import time

params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "learning_rate": 0.1,
    "num_leaves": 64,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
    # "device": "gpu",
}

lgb_tr = lgb.Dataset(X_tr, label=y_tr, free_raw_data=False)
lgb_va = lgb.Dataset(X_va, label=y_va, reference=lgb_tr, free_raw_data=False)

t0 = time.perf_counter()
model = lgb.train(
    params,
    lgb_tr,
    num_boost_round=1000,
    valid_sets=[lgb_va],
    callbacks=[lgb.early_stopping(100), lgb.log_evaluation(50)]
)
print(f"LightGBM обучался {time.perf_counter()-t0:.2f} c, best_iter =", model.best_iteration)


[LightGBM] [Info] Number of positive: 5432663, number of negative: 26654371
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.399629 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 589
[LightGBM] [Info] Number of data points in the train set: 32087034, number of used features: 60
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.169310 -> initscore=-1.590524
[LightGBM] [Info] Start training from score -1.590524
Training until validation scores don't improve for 100 rounds
[50]	valid_0's binary_logloss: 0.42877
[100]	valid_0's binary_logloss: 0.428194
[150]	valid_0's binary_logloss: 0.427182
[200]	valid_0's binary_logloss: 0.426687
[250]	valid_0's binary_logloss: 0.426539
[300]	valid_0's binary_logloss: 0.427236
Early stopping, best iteration is:
[225]	valid_0's binary_logloss: 0.426025
LightGBM обучался 430.57 c, best_iter = 225


In [ ]:
from sklearn.metrics import log_loss, roc_auc_score

p_va_lgb = model.predict(X_va, num_iteration=model.best_iteration)
print("LightGBM VAL logloss:", f"{log_loss(y_va, p_va_lgb):.6f}")
print("LightGBM VAL ROC-AUC:", f"{roc_auc_score(y_va, p_va_lgb):.6f}")

LightGBM VAL logloss: 0.426025
LightGBM VAL ROC-AUC: 0.672475


# MLP + lightgbm

In [1]:
import gc, time, math
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Sampler
from torch import amp

from tqdm.auto import tqdm
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from scipy.special import expit


pd.options.mode.chained_assignment = None
torch.backends.cudnn.benchmark = True 
t0 = time.perf_counter()

df = pd.read_csv("train.csv")

# =========================
# Время/календарь
# =========================
df["hour_dt"]     = pd.to_datetime(df["hour"].astype(str), format="%y%m%d%H", utc=True)
df["weekday"]     = df["hour_dt"].dt.weekday.astype("int8")
df["hour_of_day"] = df["hour_dt"].dt.hour.astype("int8")
df["hour_week"]   = (df["weekday"].astype("int16")*24 + df["hour_of_day"].astype("int16")).astype("int16")
df["hod_sin"]     = np.sin(2*np.pi*df["hour_of_day"]/24).astype("float32")
df["hod_cos"]     = np.cos(2*np.pi*df["hour_of_day"]/24).astype("float32")
df["wkd_sin"]     = np.sin(2*np.pi*df["weekday"]/7).astype("float32")
df["wkd_cos"]     = np.cos(2*np.pi*df["weekday"]/7).astype("float32")

# =========================
# Временной сплит: valid = последний день
# =========================
dates = np.sort(df["hour_dt"].dt.date.unique())
if len(dates) < 2:
    raise ValueError("Нужно >= 2 уникальных дат для валидации.")
last_day = dates[-1]
first_day = dates[0]
mask_val = (df["hour_dt"].dt.date == last_day)
mask_trn = ~mask_val
mask_drop_first = (df["hour_dt"].dt.date == first_day)

y = df["click"].astype(np.uint8).values
y_tr = y[mask_trn & ~mask_drop_first]
y_va = y[mask_val]

# =========================
# CTR (time-aware, бета-сглаживание)
# =========================
alpha_ctr = 5.0
cols_ctr = [c for c in ["C15","C16","C17","C18","C19","C20","C21"] if c in df.columns]
for c in cols_ctr:
    df[f"{c}_ctr"] = np.nan

p_global_all  = df.loc[mask_trn, "click"].mean()
p_global_past = df.loc[mask_trn & ~mask_drop_first, "click"].mean()

# валидный день
for c in cols_ctr:
    past = df.loc[mask_trn & ~mask_drop_first, [c, "click"]]
    agg = past.groupby(c, observed=True)["click"].agg(["count","sum"])
    smooth = (agg["sum"] + alpha_ctr*p_global_past) / (agg["count"] + alpha_ctr)
    df.loc[mask_val, f"{c}_ctr"] = df.loc[mask_val, c].map(smooth.to_dict()).fillna(p_global_past)

# прошлые дни
dates_sorted = np.sort(df["hour_dt"].dt.date.unique())
for i, d in enumerate(dates_sorted):
    if i == 0: 
        continue
    mask_day  = (df["hour_dt"].dt.date == d) & ~mask_val
    mask_prev = (df["hour_dt"].dt.date <  d) & ~mask_val
    if not mask_day.any():
        continue
    p_prev = df.loc[mask_prev, "click"].mean()
    for c in cols_ctr:
        prev = df.loc[mask_prev, [c, "click"]]
        agg  = prev.groupby(c, observed=True)["click"].agg(["count","sum"])
        smooth = (agg["sum"] + alpha_ctr*p_prev) / (agg["count"] + alpha_ctr)
        df.loc[mask_day, f"{c}_ctr"] = df.loc[mask_day, c].map(smooth.to_dict()).fillna(p_prev)

for c in cols_ctr:
    df[f"{c}_ctr"] = df[f"{c}_ctr"].fillna(p_global_all).astype("float32")

gc.collect()

# =========================
# Cumcount (train only) + logcount
# =========================
def cumcount_train_only(df_in, col, time_col="hour_dt"):
    tmp = df_in[[col, time_col]].copy()
    tmp["_idx"] = df_in.index
    tmp.sort_values(time_col, inplace=True)
    cc = tmp.groupby(col, sort=False, observed=True).cumcount().astype("int32")
    out = np.zeros(len(df_in), dtype=np.float32)
    out[tmp["_idx"].values] = cc.values
    del tmp, cc
    gc.collect()
    return out

for c in cols_ctr:
    df[f"{c}_cumcount"] = cumcount_train_only(df, c, "hour_dt").astype("float32")

solo_high = [c for c in ["site_id","site_domain","app_id","app_domain","device_model"] if c in df.columns]
for c in solo_high:
    cc = cumcount_train_only(df, c, "hour_dt").astype("float32")
    df[f"{c}_cumcount"] = cc
    df[f"{c}_logcount"] = np.log1p(cc).astype("float32")

gc.collect()

# =========================
# OOF-CTR low-card (time-aware)
# =========================
def add_oof_ctr_train_only(df_in, col, alpha=5.0):
    out_col = f"{col}_ctr_oof"
    df_in[out_col] = np.nan

    past = df_in.loc[mask_trn & ~mask_drop_first, [col, "click"]]
    agg  = past.groupby(col, observed=True)["click"].agg(["count", "sum"])
    p_gl = df_in.loc[mask_trn & ~mask_drop_first, "click"].mean()
    ctr_map_valid = (agg["sum"] + alpha * p_gl) / (agg["count"] + alpha)
    df_in.loc[mask_val, out_col] = df_in.loc[mask_val, col].map(ctr_map_valid.to_dict()).fillna(p_gl)

    for i, d in enumerate(dates_sorted):
        if i == 0:
            continue
        mask_day  = (df_in["hour_dt"].dt.date == d) & ~mask_val
        mask_prev = (df_in["hour_dt"].dt.date <  d) & ~mask_val
        if not mask_day.any():
            continue
        p_prev = df_in.loc[mask_prev, "click"].mean()
        prev = df_in.loc[mask_prev, [col, "click"]]
        agg_prev = prev.groupby(col, observed=True)["click"].agg(["count", "sum"])
        ctr_map = (agg_prev["sum"] + alpha * p_prev) / (agg_prev["count"] + alpha)
        df_in.loc[mask_day, out_col] = df_in.loc[mask_day, col].map(ctr_map.to_dict()).fillna(p_prev)

    df_in[out_col] = df_in[out_col].astype("float32", copy=False)
    gc.collect()
    return out_col

low_card_for_ctr = [c for c in ["site_category","app_category","banner_pos","C1"] if c in df.columns]
ctr_low_cols = [add_oof_ctr_train_only(df, c, alpha=5.0) for c in low_card_for_ctr]
gc.collect()

# =========================
# Финальный список фич
# =========================
base_cat_cols = [c for c in ["banner_pos","device_type","device_conn_type","C1"] if c in df.columns]
time_num_cols = [c for c in ["hour_week","hod_sin","hod_cos","wkd_sin","wkd_cos"] if c in df.columns]

num_cols = []
num_cols += [f"{c}_ctr" for c in cols_ctr]
num_cols += [f"{c}_cumcount" for c in cols_ctr]
num_cols += [f"{c}_cumcount" for c in solo_high]
num_cols += [f"{c}_logcount"  for c in solo_high]
num_cols += ctr_low_cols
num_cols += time_num_cols

# =========================
# Санитация числовых фич — никаких NaN
# =========================
def sanitize_numeric_features(df_whole: pd.DataFrame, cols: list, default_ctr: float):
    for c in cols:
        if c.endswith("_ctr") or c.endswith("_ctr_oof"):
            df_whole[c] = df_whole[c].astype("float32", copy=False).fillna(default_ctr)
        elif c.endswith("_cumcount") or c.endswith("_logcount") or c in ["hour_week","hod_sin","hod_cos","wkd_sin","wkd_cos"]:
            df_whole[c] = df_whole[c].astype("float32", copy=False).fillna(0.0)
        else:
            df_whole[c] = df_whole[c].astype("float32", copy=False).fillna(0.0)

sanitize_numeric_features(df, num_cols, default_ctr=p_global_all)
assert not df[num_cols].isna().any().any(), "NaN в числовых фичах после санитации"

# маски
use_mask_tr = mask_trn & ~mask_drop_first
use_mask_va = mask_val

# =========================
# Фреймы для LGBM (категории кодами + numeric)
# =========================
train_df = pd.DataFrame(index=df.index[use_mask_tr])
valid_df = pd.DataFrame(index=df.index[use_mask_va])

for c in base_cat_cols:
    codes = df[c].astype("category").cat.codes.astype("int32")
    train_df[c] = codes[use_mask_tr].values
    valid_df[c] = codes[use_mask_va].values
cat_indices = [train_df.columns.get_loc(c) for c in base_cat_cols]

for c in num_cols:
    train_df[c] = df.loc[use_mask_tr, c].astype("float32").values
    valid_df[c] = df.loc[use_mask_va, c].astype("float32").values

y_tr = df.loc[use_mask_tr, "click"].astype(np.uint8).values
y_va = df.loc[use_mask_va, "click"].astype(np.uint8).values

# =========================
# Подготовка данных для MLP
# =========================
emb_cols_all = ["site_id","app_id","device_model","C15","C16","C17","C18","C19","C20","C21"]
emb_cols = [c for c in emb_cols_all if c in df.columns]

id_mappings = {}
for c in emb_cols:
    cats = pd.Series(df.loc[use_mask_tr, c].astype(str).unique())
    id_mappings[c] = {cat: i+1 for i, cat in enumerate(cats)}

emb_arrays_full = {}
for c in emb_cols:
    emb_arrays_full[c] = df[c].astype(str).map(id_mappings[c]).fillna(0).astype("int64").values

# Числовые фичи для MLP
num_cols_mlp = []
for c in num_cols:
    if c.endswith("_cumcount"):
        base = c[:-9]  
        logc = f"{base}_logcount"
        if logc in df.columns:
            if logc not in num_cols_mlp:
                num_cols_mlp.append(logc)
        else:
            tmp_col = f"{c}__log1p_tmp"
            df[tmp_col] = np.log1p(df[c]).astype("float32")
            num_cols_mlp.append(tmp_col)
    else:
        if c not in num_cols_mlp:
            num_cols_mlp.append(c)

# Массивы для MLP
X_num_full = df[num_cols_mlp].astype("float32").values
emb_tr = [emb_arrays_full[c][use_mask_tr] for c in emb_cols]
emb_va = [emb_arrays_full[c][use_mask_va] for c in emb_cols]
X_num_tr = X_num_full[use_mask_tr]
X_num_va = X_num_full[use_mask_va]

assert not np.isnan(X_num_tr).any(), "NaN в X_num_tr"
assert not np.isnan(X_num_va).any(), "NaN в X_num_va"

emb_sizes = []
for c in emb_cols:
    n_tok = len(id_mappings[c]) + 1
    dim = int(min(32, max(4, round(n_tok**0.25 * 8))))
    emb_sizes.append((n_tok, dim))

# =========================
# Torch Dataset + BalancedBatchSampler + OneCycleLR + AMP
# =========================
class CTRDataset(Dataset):
    def __init__(self, emb_arrays, num_array, y=None):
        self.emb_arrays = emb_arrays
        self.num_array = num_array
        self.y = y
    def __len__(self):
        return self.num_array.shape[0]
    def __getitem__(self, idx):
        cats = [torch.tensor(a[idx], dtype=torch.long) for a in self.emb_arrays]
        nums = torch.tensor(self.num_array[idx], dtype=torch.float32)
        if self.y is None:
            return cats, nums
        else:
            return cats, nums, torch.tensor(self.y[idx], dtype=torch.float32)

class BalancedBatchSampler(Sampler):
    def __init__(self, y_array, batch_size, pos_frac, num_batches, seed=42):
        self.y = y_array
        self.batch_size = int(batch_size)
        self.pos_bs = max(1, int(round(batch_size * pos_frac)))
        self.neg_bs = self.batch_size - self.pos_bs
        self.num_batches = int(num_batches)
        self.rng = np.random.default_rng(seed)
        self.pos_idx = np.where(self.y == 1)[0]
        self.neg_idx = np.where(self.y == 0)[0]
        if len(self.pos_idx) == 0 or len(self.neg_idx) == 0:
            raise ValueError("Нет положительных или отрицательных примеров.")
    def __iter__(self):
        for _ in range(self.num_batches):
            pos = self.rng.choice(self.pos_idx, size=self.pos_bs, replace=True)
            neg = self.rng.choice(self.neg_idx, size=self.neg_bs, replace=True)
            idx = np.concatenate([pos, neg])
            self.rng.shuffle(idx)
            yield idx.tolist()
    def __len__(self):
        return self.num_batches

tr_dataset = CTRDataset(emb_tr, X_num_tr, y_tr)
va_dataset = CTRDataset(emb_va, X_num_va, y_va)

/home/misha/lgbm_cuda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
target_pos_frac = 0.25
bs_train = 16384

epoch_samples = min(len(y_tr), 5_000_000)
num_batches = math.ceil(epoch_samples / bs_train)

balanced_sampler = BalancedBatchSampler(y_tr, batch_size=bs_train, pos_frac=target_pos_frac, num_batches=num_batches)
tr_loader = DataLoader(tr_dataset, batch_sampler=balanced_sampler,
                       num_workers=4, prefetch_factor=1, pin_memory=True, persistent_workers=True)
va_loader = DataLoader(va_dataset, batch_size=32768, shuffle=False,
                       num_workers=2, prefetch_factor=1, pin_memory=True, persistent_workers=True)

# =========================
# Модель MLP
# =========================
class MLPEmbModel(nn.Module):
    def __init__(self, emb_sizes, num_dim, hidden_dims=[128,64], dropout=0.2):
        super().__init__()
        self.emb_layers = nn.ModuleList([nn.Embedding(n, d) for n, d in emb_sizes])
        emb_total = sum(d for _, d in emb_sizes)
        in_dim = emb_total + num_dim
        layers = []
        last = in_dim
        for h in hidden_dims:
            layers += [nn.Linear(last, h), nn.ReLU(), nn.Dropout(dropout)]
            last = h
        self.mlp = nn.Sequential(*layers)
        self.out = nn.Linear(last, 1)
    def forward(self, cats, nums):
        if len(self.emb_layers) > 0:
            emb_outs = [emb(c) for emb, c in zip(self.emb_layers, cats)]
            x = torch.cat(emb_outs + [nums], dim=1)
        else:
            x = nums
        h = self.mlp(x) if len(self.mlp) > 0 else x
        logit = self.out(h)
        return logit, h

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLPEmbModel(emb_sizes=emb_sizes, num_dim=X_num_tr.shape[1], hidden_dims=[128,64], dropout=0.2).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

num_epochs = 3
total_steps = num_epochs * len(tr_loader)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=1e-3, total_steps=total_steps,
    pct_start=0.1, div_factor=10, final_div_factor=100
)
scaler = amp.GradScaler('cuda', enabled=(device.type == "cuda"))

# =========================
# Обучение MLP (AMP + clip grad + OneCycleLR)
# =========================
for ep in range(1, num_epochs + 1):
    model.train()
    running = 0.0
    for cats, nums, tgt in tqdm(tr_loader, desc=f"Epoch {ep}/{num_epochs} [train]"):
        cats = [c.to(device, non_blocking=True) for c in cats]
        nums = nums.to(device, non_blocking=True)
        tgt  = tgt.to(device, non_blocking=True).view(-1,1).float()

        optimizer.zero_grad(set_to_none=True)
        with amp.autocast('cuda', enabled=(device.type == "cuda")):
            logit, _ = model(cats, nums)
        loss = criterion(logit.float(), tgt)

        if not torch.isfinite(loss):
            continue

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        running += float(loss.detach().cpu())

    print(f"[MLP] Epoch {ep} train loss: {running/max(1,len(tr_loader)):.4f}")

    # Валидация
    model.eval()
    val_preds, val_tgts = [], []
    with torch.no_grad():
        for cats, nums, tgt in tqdm(va_loader, desc=f"Epoch {ep}/{num_epochs} [valid]"):
            cats = [c.to(device, non_blocking=True) for c in cats]
            nums = nums.to(device, non_blocking=True)
            with amp.autocast('cuda', enabled=(device.type == "cuda")):
                logit, _ = model(cats, nums)
            prob = torch.sigmoid(logit.float()).squeeze(1)
            prob = torch.nan_to_num(prob, nan=0.5, posinf=1.0, neginf=0.0)
            val_preds.append(prob.cpu().numpy())
            val_tgts.append(tgt.numpy())

    val_preds = np.concatenate(val_preds)
    val_tgts  = np.concatenate(val_tgts)
    if np.isnan(val_preds).any():
        val_preds = np.nan_to_num(val_preds, nan=0.5, posinf=1.0, neginf=0.0)
    va_auc = roc_auc_score(val_tgts, val_preds)
    print(f"[MLP] Epoch {ep}  Val AUC: {va_auc:.4f}")


Epoch 1/3 [train]: 100%|██████████| 306/306 [01:13<00:00,  4.14it/s]


[MLP] Epoch 1 train loss: 0.5177


Epoch 1/3 [valid]: 100%|██████████| 129/129 [01:39<00:00,  1.30it/s]


[MLP] Epoch 1  Val AUC: 0.7230


Epoch 2/3 [train]: 100%|██████████| 306/306 [01:11<00:00,  4.28it/s]


...

[MLP] Epoch 2  Val AUC: 0.7280


Epoch 3/3 [train]: 100%|██████████| 306/306 [01:10<00:00,  4.33it/s]


[MLP] Epoch 3 train loss: 0.4927


Epoch 3/3 [valid]: 100%|██████████| 129/129 [01:36<00:00,  1.33it/s]


[MLP] Epoch 3  Val AUC: 0.7290


In [3]:
# =========================
# Извлекаем вероятности MLP (экономно по памяти)
# =========================
from torch import amp
from tqdm.auto import tqdm

def extract_probs(ds, model, device, bs=16384):
    dl = DataLoader(
        ds,
        batch_size=bs,
        shuffle=False,
        num_workers=0,          
        pin_memory=False,     
        prefetch_factor=None,   
        persistent_workers=False
    )

    model.eval()
    n = len(ds)
    out = np.empty(n, dtype=np.float16) 
    idx = 0
    with torch.inference_mode():       
        for cats, nums in tqdm(dl, desc="Extract MLP probs"):
            bsz = nums.shape[0]
            cats = [c.to(device, non_blocking=False) for c in cats]
            nums = nums.to(device, non_blocking=False)
            with amp.autocast('cuda', enabled=(device.type == "cuda")):
                logit, _ = model(cats, nums)
            prob = torch.sigmoid(logit).squeeze(1)

            prob = torch.nan_to_num(prob, nan=0.5, posinf=1.0, neginf=0.0)
            out[idx:idx+bsz] = prob.detach().cpu().to(torch.float16).numpy()
            idx += bsz

    # небольшая уборка
    del dl
    if device.type == "cuda":
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
    gc.collect()
    return out

# инференс вероятностей
tr_infer_ds = CTRDataset(emb_tr, X_num_tr, None)
va_infer_ds = CTRDataset(emb_va, X_num_va, None)
tr_probs = extract_probs(tr_infer_ds, model, device, bs=16384)   
va_probs = extract_probs(va_infer_ds, model, device, bs=16384)

mlp_val_auc = roc_auc_score(y_va, va_probs.astype(np.float32))
print(f"[MLP] Final Val AUC = {mlp_val_auc:.4f}")

del tr_loader, va_loader, tr_dataset, va_dataset, tr_infer_ds, va_infer_ds
if device.type == "cuda":
    torch.cuda.synchronize()
    torch.cuda.empty_cache()
gc.collect()


Extract MLP probs: 100%|██████████| 258/258 [02:32<00:00,  1.69it/s]


[MLP] Final Val AUC = 0.7290


0

In [4]:
# =========================
# LGBM с init_score от MLP
# =========================

safe_ctr = [c for c in train_df.columns if c.endswith("_ctr") and not c.endswith("_ctr_oof")]
oof_ctr  = [c for c in train_df.columns if c.endswith("_ctr_oof")]
logcnt   = [c for c in train_df.columns if c.endswith("_logcount")]
time_num_present = [c for c in time_num_cols if c in train_df.columns]

num_cols_lgb = safe_ctr + oof_ctr + logcnt + time_num_present
num_cols_lgb = [c for c in num_cols_lgb if not c.endswith("_cumcount")]

lgb_train_df = pd.DataFrame(index=train_df.index)
lgb_valid_df = pd.DataFrame(index=valid_df.index)

for c in base_cat_cols:
    lgb_train_df[c] = train_df[c].astype("int32")
    lgb_valid_df[c] = valid_df[c].astype("int32")
cat_indices = [lgb_train_df.columns.get_loc(c) for c in base_cat_cols]

for c in num_cols_lgb:
    lgb_train_df[c] = train_df[c].astype("float32")
    lgb_valid_df[c] = valid_df[c].astype("float32")

def probs_to_logit(p):
    p = np.asarray(p, dtype=np.float32)
    p = np.clip(p, 1e-6, 1-1e-6)
    return np.log(p/(1-p)).astype(np.float32)

tr_logit = probs_to_logit(tr_probs)
va_logit = probs_to_logit(va_probs)

del train_df, valid_df
gc.collect()

dtrain = lgb.Dataset(lgb_train_df, label=y_tr,
                     categorical_feature=cat_indices, free_raw_data=True)
dvalid = lgb.Dataset(lgb_valid_df, label=y_va,
                     categorical_feature=cat_indices, reference=dtrain, free_raw_data=True)

dtrain.set_init_score(tr_logit)
dvalid.set_init_score(va_logit)

del lgb_train_df, lgb_valid_df, tr_logit, va_logit
gc.collect()

params = dict(
    objective="binary",
    metric="auc",
    learning_rate=0.02,
    num_leaves=255,
    max_depth=-1,
    min_data_in_leaf=50,
    feature_fraction=0.70,
    bagging_fraction=0.70,
    bagging_freq=1,
    lambda_l2=1.0,
    max_bin=63, 
    bin_construct_sample_cnt=100000,
    device_type="cuda",
    gpu_device_id=0,
    verbosity=-1,
)

print("[LGBM] Training on GPU with init_score (memory-friendly)...")
callbacks = [
    lgb.early_stopping(stopping_rounds=200, first_metric_only=True),
    lgb.log_evaluation(period=100),
]

booster = lgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    valid_sets=[dvalid],
    valid_names=["valid"],
    callbacks=callbacks
)


[LGBM] Training on GPU with init_score (memory-friendly)...
Training until validation scores don't improve for 200 rounds
[100]	valid's auc: 0.731453
[200]	valid's auc: 0.732525
[300]	valid's auc: 0.732944
[400]	valid's auc: 0.733358
[500]	valid's auc: 0.733622
[600]	valid's auc: 0.73369
[700]	valid's auc: 0.73363
[800]	valid's auc: 0.733819
[900]	valid's auc: 0.733763
Early stopping, best iteration is:
[784]	valid's auc: 0.733855
Evaluated only: auc


# RAT

In [ ]:
import gc, time, math
from collections import defaultdict, deque

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Sampler
from torch import amp

from tqdm.auto import tqdm
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from scipy.special import expit 

pd.options.mode.chained_assignment = None
torch.backends.cudnn.benchmark = True
t0 = time.perf_counter()

df = pd.read_csv("train.csv")

# =========================
# Время/календарь
# =========================
df["hour_dt"]     = pd.to_datetime(df["hour"].astype(str), format="%y%m%d%H", utc=True)
df["weekday"]     = df["hour_dt"].dt.weekday.astype("int8")
df["hour_of_day"] = df["hour_dt"].dt.hour.astype("int8")
df["hour_week"]   = (df["weekday"].astype("int16")*24 + df["hour_of_day"].astype("int16")).astype("int16")
df["hod_sin"]     = np.sin(2*np.pi*df["hour_of_day"]/24).astype("float32")
df["hod_cos"]     = np.cos(2*np.pi*df["hour_of_day"]/24).astype("float32")
df["wkd_sin"]     = np.sin(2*np.pi*df["weekday"]/7).astype("float32")
df["wkd_cos"]     = np.cos(2*np.pi*df["weekday"]/7).astype("float32")

# =========================
# Временной сплит: valid = последний день
# =========================
dates = np.sort(df["hour_dt"].dt.date.unique())
if len(dates) < 2:
    raise ValueError("Нужно >= 2 уникальных дат для валидации.")
last_day  = dates[-1]
first_day = dates[0]
mask_val = (df["hour_dt"].dt.date == last_day)
mask_trn = ~mask_val
mask_drop_first = (df["hour_dt"].dt.date == first_day)

y = df["click"].astype(np.uint8).values
y_tr = y[mask_trn & ~mask_drop_first]
y_va = y[mask_val]

# =========================
# CTR (time-aware, бета-сглаживание)
# =========================
alpha_ctr = 5.0
cols_ctr = [c for c in ["C15","C16","C17","C18","C19","C20","C21"] if c in df.columns]
for c in cols_ctr:
    df[f"{c}_ctr"] = np.nan

p_global_all  = df.loc[mask_trn, "click"].mean()
p_global_past = df.loc[mask_trn & ~mask_drop_first, "click"].mean()

# валидный день
for c in cols_ctr:
    past = df.loc[mask_trn & ~mask_drop_first, [c, "click"]]
    agg = past.groupby(c, observed=True)["click"].agg(["count","sum"])
    smooth = (agg["sum"] + alpha_ctr*p_global_past) / (agg["count"] + alpha_ctr)
    df.loc[mask_val, f"{c}_ctr"] = df.loc[mask_val, c].map(smooth.to_dict()).fillna(p_global_past)

# прошлые дни
dates_sorted = np.sort(df["hour_dt"].dt.date.unique())
for i, d in enumerate(dates_sorted):
    if i == 0:
        continue
    mask_day  = (df["hour_dt"].dt.date == d) & ~mask_val
    mask_prev = (df["hour_dt"].dt.date <  d) & ~mask_val
    if not mask_day.any():
        continue
    p_prev = df.loc[mask_prev, "click"].mean()
    for c in cols_ctr:
        prev = df.loc[mask_prev, [c, "click"]]
        agg  = prev.groupby(c, observed=True)["click"].agg(["count","sum"])
        smooth = (agg["sum"] + alpha_ctr*p_prev) / (agg["count"] + alpha_ctr)
        df.loc[mask_day, f"{c}_ctr"] = df.loc[mask_day, c].map(smooth.to_dict()).fillna(p_prev)

for c in cols_ctr:
    df[f"{c}_ctr"] = df[f"{c}_ctr"] .fillna(p_global_all).astype("float32")

gc.collect()

# =========================
# Cumcount (train only) + logcount
# =========================
def cumcount_train_only(df_in, col, time_col="hour_dt"):
    tmp = df_in[[col, time_col]].copy()
    tmp["_idx"] = df_in.index
    tmp.sort_values(time_col, inplace=True)
    cc = tmp.groupby(col, sort=False, observed=True).cumcount().astype("int32")
    out = np.zeros(len(df_in), dtype=np.float32)
    out[tmp["_idx"].values] = cc.values
    del tmp, cc
    gc.collect()
    return out

for c in cols_ctr:
    df[f"{c}_cumcount"] = cumcount_train_only(df, c, "hour_dt").astype("float32")

solo_high = [c for c in ["site_id","site_domain","app_id","app_domain","device_model"] if c in df.columns]
for c in solo_high:
    cc = cumcount_train_only(df, c, "hour_dt").astype("float32")
    df[f"{c}_cumcount"] = cc
    df[f"{c}_logcount"] = np.log1p(cc).astype("float32")

gc.collect()

# =========================
# OOF-CTR low-card (time-aware)
# =========================
def add_oof_ctr_train_only(df_in, col, alpha=5.0):
    out_col = f"{col}_ctr_oof"
    df_in[out_col] = np.nan

    past = df_in.loc[mask_trn & ~mask_drop_first, [col, "click"]]
    agg  = past.groupby(col, observed=True)["click"].agg(["count", "sum"])
    p_gl = df_in.loc[mask_trn & ~mask_drop_first, "click"].mean()
    ctr_map_valid = (agg["sum"] + alpha * p_gl) / (agg["count"] + alpha)
    df_in.loc[mask_val, out_col] = df_in.loc[mask_val, col].map(ctr_map_valid.to_dict()).fillna(p_gl)

    for i, d in enumerate(dates_sorted):
        if i == 0:
            continue
        mask_day  = (df_in["hour_dt"].dt.date == d) & ~mask_val
        mask_prev = (df_in["hour_dt"].dt.date <  d) & ~mask_val
        if not mask_day.any():
            continue
        p_prev = df_in.loc[mask_prev, "click"].mean()   
        prev = df_in.loc[mask_prev, [col, "click"]]
        agg_prev = prev.groupby(col, observed=True)["click"].agg(["count", "sum"])
        ctr_map = (agg_prev["sum"] + alpha * p_prev) / (agg_prev["count"] + alpha)
        df_in.loc[mask_day, out_col] = df_in.loc[mask_day, col].map(ctr_map.to_dict()).fillna(p_prev)

    df_in[out_col] = df_in[out_col].astype("float32", copy=False)
    gc.collect()
    return out_col

low_card_for_ctr = [c for c in ["site_category","app_category","banner_pos","C1"] if c in df.columns]
ctr_low_cols = [add_oof_ctr_train_only(df, c, alpha=5.0) for c in low_card_for_ctr]
gc.collect()

# =========================
# Финальный список фич
# =========================
base_cat_cols = [c for c in ["banner_pos","device_type","device_conn_type","C1"] if c in df.columns]
time_num_cols = [c for c in ["hour_week","hod_sin","hod_cos","wkd_sin","wkd_cos"] if c in df.columns]

num_cols = []
num_cols += [f"{c}_ctr" for c in cols_ctr]
num_cols += [f"{c}_cumcount" for c in cols_ctr]
num_cols += [f"{c}_cumcount" for c in solo_high]
num_cols += [f"{c}_logcount"  for c in solo_high]
num_cols += ctr_low_cols
num_cols += time_num_cols

# =========================
# Санитация числовых фич — никаких NaN
# =========================
def sanitize_numeric_features(df_whole: pd.DataFrame, cols: list, default_ctr: float):
    for c in cols:
        if c.endswith("_ctr") or c.endswith("_ctr_oof"):
            df_whole[c] = df_whole[c].astype("float32", copy=False).fillna(default_ctr)
        elif c.endswith("_cumcount") or c.endswith("_logcount") or c in ["hour_week","hod_sin","hod_cos","wkd_sin","wkd_cos"]:
            df_whole[c] = df_whole[c].astype("float32", copy=False).fillna(0.0)
        else:
            df_whole[c] = df_whole[c].astype("float32", copy=False).fillna(0.0)

sanitize_numeric_features(df, num_cols, default_ctr=p_global_all)
assert not df[num_cols].isna().any().any(), "NaN в числовых фичах после санитации"

# маски
use_mask_tr = mask_trn & ~mask_drop_first
use_mask_va = mask_val

# =========================
# Фреймы для LGBM (категории кодами + numeric)
# =========================
train_df = pd.DataFrame(index=df.index[use_mask_tr])
valid_df = pd.DataFrame(index=df.index[use_mask_va])

for c in base_cat_cols:
    codes = df[c].astype("category").cat.codes.astype("int32")
    train_df[c] = codes[use_mask_tr].values
    valid_df[c] = codes[use_mask_va].values
cat_indices = [train_df.columns.get_loc(c) for c in base_cat_cols]

for c in num_cols:
    train_df[c] = df.loc[use_mask_tr, c].astype("float32").values
    valid_df[c] = df.loc[use_mask_va, c].astype("float32").values

y_tr = df.loc[use_mask_tr, "click"].astype(np.uint8).values
y_va = df.loc[use_mask_va, "click"].astype(np.uint8).values

# =========================
# Подготовка данных для эмбеддингов 
# =========================
emb_cols_all = ["site_id","app_id","device_model","C15","C16","C17","C18","C19","C20","C21"]
emb_cols = [c for c in emb_cols_all if c in df.columns]

emb_arrays_full = {}
n_tokens_per_col = {}
for c in emb_cols:
    train_cats = pd.Categorical(df.loc[use_mask_tr, c]).categories
    cat_all = pd.Categorical(df[c], categories=train_cats)

    codes_i32 = cat_all.codes.astype(np.int32, copy=False)  
    codes_i32 += 1                                      
    emb_arrays_full[c] = codes_i32.astype(np.uint32, copy=False)

    n_tokens_per_col[c] = len(train_cats) + 1 


num_cols_mlp = []
_seen = set()
for c in num_cols:
    if c.endswith("_cumcount"):
        base = c[:-9]
        logc = f"{base}_logcount"
        key = logc if logc in df.columns else c
    else:
        key = c
    if key not in _seen:
        num_cols_mlp.append(key)
        _seen.add(key)

def _col_values_for(mask: pd.Series, col: str) -> np.ndarray:
    if col.endswith("_cumcount") and f"{col[:-9]}_logcount" not in df.columns:
        return np.log1p(df.loc[mask, col].to_numpy(dtype=np.float32, copy=False))
    else:
        return df.loc[mask, col].to_numpy(dtype=np.float32, copy=False)

tr_cols, va_cols = [], []
for col in num_cols_mlp:
    tr_cols.append(_col_values_for(use_mask_tr, col).reshape(-1, 1))
    va_cols.append(_col_values_for(use_mask_va, col).reshape(-1, 1))

X_num_tr = (np.concatenate(tr_cols, axis=1).astype(np.float16, copy=False)
            if tr_cols else np.zeros((use_mask_tr.sum(), 0), np.float16))
X_num_va = (np.concatenate(va_cols, axis=1).astype(np.float16, copy=False)
            if va_cols else np.zeros((use_mask_va.sum(), 0), np.float16))

emb_tr = [emb_arrays_full[c][use_mask_tr] for c in emb_cols]
emb_va = [emb_arrays_full[c][use_mask_va] for c in emb_cols]

assert not np.isnan(X_num_tr).any(), "NaN в X_num_tr"
assert not np.isnan(X_num_va).any(), "NaN в X_num_va"

emb_sizes = []
for c in emb_cols:
    n_tok = n_tokens_per_col[c]
    dim = int(min(32, max(4, round(n_tok**0.25 * 8))))
    emb_sizes.append((n_tok, dim))

_df_keep = ["hour_dt"] + emb_cols
_drop = [c for c in list(df.columns) if c not in _df_keep]
if _drop:
    df.drop(columns=_drop, inplace=True)

del tr_cols, va_cols, _seen
gc.collect()

for c in base_cat_cols:
    if str(train_df[c].dtype).startswith("int") and train_df[c].dtype.itemsize > 2:
        train_df[c] = train_df[c].astype("int16")
        valid_df[c] = valid_df[c].astype("int16")
gc.collect()


/home/misha/lgbm_cuda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0

12 min

In [2]:
# =========================
# RAT: Retrieval-Augmented Transformer
# =========================
RAT_K = 4
RAT_MAX_PER_TOKEN = 256
RAT_D_MODEL = 32
RAT_NUM_BLOCKS = 2
RAT_NUM_HEADS = 4
RAT_DROPOUT = 0.1

def _value_counts_map(series: pd.Series) -> dict:
    vc = series.value_counts(dropna=False)
    return {k: int(v) for k, v in vc.items()}

# =========================
# БЫСТРАЯ версия ретрива (векторизованная)
# =========================
def build_retrieval_indices_fast(
    df_time_values,
    emb_arrays_full: dict[str, np.ndarray],
    emb_cols: list[str],
    use_mask_tr: pd.Series,
    use_mask_va: pd.Series,
    K: int = 4,
    max_per_token: int = 256,
    top_fields: int | None = 3,
):
    pos = np.arange(df_time_values.shape[0])
    pos_tr = pos[use_mask_tr.values]
    pos_va = pos[use_mask_va.values]

    t_tr = df_time_values[pos_tr].view("int64")
    order_tr = np.lexsort((pos_tr, t_tr))
    pos_tr_sorted = pos_tr[order_tr]
    n_tr = len(pos_tr_sorted)
    n_va = len(pos_va)
    if n_tr == 0:
        return np.empty((0, K), dtype=np.int32), np.empty((n_va, K), dtype=np.int32)

    codes_tr = {c: emb_arrays_full[c][use_mask_tr.values][order_tr].astype(np.int32, copy=False) for c in emb_cols}
    codes_va = {c: emb_arrays_full[c][use_mask_va.values].astype(np.int32, copy=False) for c in emb_cols}

    N = float(n_tr)
    idf_map = {}
    for c in emb_cols:
        x = codes_tr[c]
        if x.size == 0:
            idf_map[c] = np.zeros(1, dtype=np.float32)
            continue
        vmax = int(x.max())
        cnt = np.bincount(x, minlength=vmax + 1).astype(np.int32)
        idf = np.zeros_like(cnt, dtype=np.float32)
        nonzero = cnt > 0
        idf[nonzero] = np.log((N - cnt[nonzero] + 0.5) / (cnt[nonzero] + 0.5)).astype(np.float32)
        idf_map[c] = idf

    from collections import deque, defaultdict
    inv_train = {c: defaultdict(lambda: deque(maxlen=max_per_token)) for c in emb_cols}
    inv_full  = {c: defaultdict(lambda: deque(maxlen=max_per_token)) for c in emb_cols}
    for i in range(n_tr):
        for c in emb_cols:
            inv_full[c][int(codes_tr[c][i])].append(i)

    retr_tr_idx = np.full((n_tr, K), -1, dtype=np.int32)
    retr_va_idx = np.full((n_va, K), -1, dtype=np.int32)

    scores = np.zeros(n_tr, dtype=np.float32)

    def _pick_topk_from_touched(touched_arr):
        if touched_arr.size == 0:
            return []
        s = scores[touched_arr]
        if s.size <= K:
            order = np.argsort(s)[::-1]
            return touched_arr[order].tolist()
        idxk = np.argpartition(s, -K)[-K:]
        order = np.argsort(s[idxk])[::-1]
        return touched_arr[idxk[order]].tolist()

    def _select_cols_for_row(codes_row: dict[str, int]):
        if top_fields is None or top_fields >= len(emb_cols):
            return emb_cols
        pairs = []
        for c in emb_cols:
            code = int(codes_row[c])
            idf = idf_map[c][code] if code < idf_map[c].shape[0] else 0.0
            pairs.append((c, idf))
        pairs.sort(key=lambda t: t[1], reverse=True)
        return [c for c, _ in pairs[:top_fields]]

    # TRAIN: только ранее виденные
    for i in range(n_tr):
        touched_chunks = []
        cols_use = _select_cols_for_row({c: codes_tr[c][i] for c in emb_cols})
        for c in cols_use:
            code = int(codes_tr[c][i])
            lst = inv_train[c][code]
            if not lst:
                continue
            w = idf_map[c][code] if code < idf_map[c].shape[0] else 0.0
            if w == 0.0:
                continue
            arr = np.fromiter(lst, dtype=np.int32)
            scores[arr] += w
            touched_chunks.append(arr)
        if touched_chunks:
            touched = np.unique(np.concatenate(touched_chunks))
            top = _pick_topk_from_touched(touched)
            if top:
                retr_tr_idx[i, :len(top)] = top[:K]
            scores[touched] = 0.0
        for c in emb_cols:
            inv_train[c][int(codes_tr[c][i])].append(i)

    for j in range(n_va):
        touched_chunks = []
        cols_use = _select_cols_for_row({c: codes_va[c][j] for c in emb_cols})
        for c in cols_use:
            code = int(codes_va[c][j])
            lst = inv_full[c][code]
            if not lst:
                continue
            w = idf_map[c][code] if code < idf_map[c].shape[0] else 0.0
            if w == 0.0:
                continue
            arr = np.fromiter(lst, dtype=np.int32)
            scores[arr] += w
            touched_chunks.append(arr)
        if touched_chunks:
            touched = np.unique(np.concatenate(touched_chunks))
            top = _pick_topk_from_touched(touched)
            if top:
                retr_va_idx[j, :len(top)] = top[:K]
            scores[touched] = 0.0

    return retr_tr_idx, retr_va_idx

In [3]:
retr_tr_idx, retr_va_idx = build_retrieval_indices_fast(
    df_time_values=df["hour_dt"].values,
    emb_arrays_full=emb_arrays_full,
    emb_cols=emb_cols,
    use_mask_tr=use_mask_tr,
    use_mask_va=use_mask_va,
    K=RAT_K,
    max_per_token=RAT_MAX_PER_TOKEN,
    top_fields=3,
)

pos = np.arange(len(df))
pos_tr = pos[use_mask_tr.values]
t_tr = df["hour_dt"].values[pos_tr].view("int64")
order_tr = np.lexsort((pos_tr, t_tr))

def _remap_sorted_to_mask(retr_idx_sorted: np.ndarray, order_tr: np.ndarray) -> np.ndarray:
    """Переводит индексы из пространства [0..n_tr) 'time-sorted' в индексы по маске train (исходный порядок)."""
    out = np.full_like(retr_idx_sorted, -1)
    m = retr_idx_sorted >= 0
    out[m] = order_tr[retr_idx_sorted[m]]
    return out

retr_tr_idx_mask = _remap_sorted_to_mask(retr_tr_idx, order_tr)
retr_va_idx_mask = _remap_sorted_to_mask(retr_va_idx, order_tr)


53 min on local maschine

In [4]:

class CTRDatasetRAT(Dataset):
    def __init__(self, emb_target, emb_pool, num_target, y_target, retr_idx, pool_labels):
        self.emb_target  = emb_target
        self.emb_pool    = emb_pool
        self.num_target  = num_target
        self.y_target    = y_target
        self.retr_idx    = retr_idx
        self.pool_labels = pool_labels
        self.F = len(emb_target)

    def __len__(self):
        return self.num_target.shape[0]

    def __getitem__(self, idx):
        K = self.retr_idx.shape[1]
        cats_all = np.zeros((K+1, self.F), dtype=np.int64)
        for f in range(self.F):
            cats_all[0, f] = int(self.emb_target[f][idx])

        retr = self.retr_idx[idx]
        valid = (retr >= 0)
        if valid.any():
            rpos = np.where(valid)[0]
            rids = retr[valid]
            for f in range(self.F):
                tmp = cats_all[1:, f]
                tmp[rpos] = self.emb_pool[f][rids]
                cats_all[1:, f] = tmp

        labs_all = np.full(K+1, 2, dtype=np.int64) 
        if valid.any():
            labs_all[1:][valid] = self.pool_labels[retr[valid]].astype(np.int64)

        nums = self.num_target[idx].astype(np.float32)

        if self.y_target is None:
            return cats_all, labs_all, nums
        else:
            y = float(self.y_target[idx])
            return cats_all, labs_all, nums, y


def rat_collate_train(batch):
    cats, labs, nums, y = zip(*batch)
    return (
        torch.tensor(np.stack(cats), dtype=torch.long),
        torch.tensor(np.stack(labs), dtype=torch.long),
        torch.tensor(np.stack(nums), dtype=torch.float32),
        torch.tensor(y, dtype=torch.float32).view(-1, 1),
    )

def rat_collate_infer(batch):
    cats, labs, nums = zip(*batch)
    return (
        torch.tensor(np.stack(cats), dtype=torch.long),
        torch.tensor(np.stack(labs), dtype=torch.long),
        torch.tensor(np.stack(nums), dtype=torch.float32),
    )


class RatBlock(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)
        self.ln3 = nn.LayerNorm(d_model)
        self.mha_intra = nn.MultiheadAttention(d_model, n_heads, dropout=dropout, batch_first=True)
        self.mha_cross = nn.MultiheadAttention(d_model, n_heads, dropout=dropout, batch_first=True)
        self.ff = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout),
        )

    def forward(self, x): 
        B, Ks, Fp, D = x.shape
        h1 = self.ln1(x).reshape(B * Ks, Fp, D)
        a1, _ = self.mha_intra(h1, h1, h1)
        x = x + a1.reshape(B, Ks, Fp, D)

        h2 = self.ln2(x).permute(0, 2, 1, 3).reshape(B * Fp, Ks, D)
        a2, _ = self.mha_cross(h2, h2, h2)
        x = x + a2.reshape(B, Fp, Ks, D).permute(0, 2, 1, 3)

        x = x + self.ff(self.ln3(x))
        return x


class RATModel(nn.Module):
    def __init__(self, emb_sizes, d_model, n_blocks, n_heads, d_ff=128, dropout=0.1, num_dim=0):
        super().__init__()
        self.F = len(emb_sizes)
        self.embs = nn.ModuleList([nn.Embedding(n, d) for (n, d) in emb_sizes])
        self.projs = nn.ModuleList([
            nn.Identity() if d == d_model else nn.Linear(d, d_model, bias=False)
            for (_, d) in emb_sizes
        ])
        self.label_emb = nn.Embedding(3, d_model)
        self.blocks = nn.ModuleList([RatBlock(d_model, n_heads, d_ff, dropout) for _ in range(n_blocks)])
        self.num_proj = nn.Identity() if num_dim == 0 else nn.Sequential(
            nn.Linear(num_dim, d_model),
            nn.ReLU(),
            nn.Dropout(dropout),
        )
        self.head = nn.Sequential(
            nn.Linear(d_model * (2 if num_dim > 0 else 1), d_model),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, 1),
        )

    def forward(self, cats_all, label_ids, nums=None):
        B, Ks, F = cats_all.shape
        feats = []
        for f in range(self.F):
            e = self.embs[f](cats_all[:, :, f]) 
            e = self.projs[f](e)                 
            feats.append(e.unsqueeze(2))           
        feat_stack = torch.cat(feats, dim=2)   
        lab = self.label_emb(label_ids).unsqueeze(2) 
        x = torch.cat([lab, feat_stack], dim=2)     
        for blk in self.blocks:
            x = blk(x)
        pooled = x[:, 0, 0, :]                    
        if nums is not None and nums.numel() > 0:
            pooled = torch.cat([pooled, self.num_proj(nums)], dim=1)
        return self.head(pooled)                   


# Балансировщик батчей
class BalancedBatchSampler(Sampler):
    def __init__(self, y_array, batch_size, pos_frac, num_batches, seed=42):
        self.y = y_array
        self.batch_size = int(batch_size)
        self.pos_bs = max(1, int(round(batch_size * pos_frac)))
        self.neg_bs = self.batch_size - self.pos_bs
        self.num_batches = int(num_batches)
        self.rng = np.random.default_rng(seed)
        self.pos_idx = np.where(self.y == 1)[0]
        self.neg_idx = np.where(self.y == 0)[0]
        if len(self.pos_idx) == 0 or len(self.neg_idx) == 0:
            raise ValueError("Нет положительных или отрицательных примеров.")

    def __iter__(self):
        for _ in range(self.num_batches):
            pos = self.rng.choice(self.pos_idx, size=self.pos_bs, replace=True)
            neg = self.rng.choice(self.neg_idx, size=self.neg_bs, replace=True)
            idx = np.concatenate([pos, neg])
            self.rng.shuffle(idx)
            yield idx.tolist()

    def __len__(self):
        return self.num_batches



tr_dataset = CTRDatasetRAT(emb_tr, emb_tr, X_num_tr, y_tr, retr_tr_idx_mask, y_tr)
va_dataset = CTRDatasetRAT(emb_va, emb_tr, X_num_va, y_va, retr_va_idx_mask, y_tr)

target_pos_frac = 0.25
bs_train = 4096                   
epoch_samples = min(len(y_tr), 5_000_000)
num_batches = math.ceil(epoch_samples / bs_train)

balanced_sampler = BalancedBatchSampler(
    y_tr, batch_size=bs_train, pos_frac=target_pos_frac, num_batches=num_batches
)

tr_loader = DataLoader(
    tr_dataset, batch_sampler=balanced_sampler,
    num_workers=0, pin_memory=False, collate_fn=rat_collate_train
)

va_loader = DataLoader(
    va_dataset, batch_size=16384, shuffle=False,
    num_workers=0, pin_memory=False, collate_fn=rat_collate_train
)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RATModel(
    emb_sizes=emb_sizes, d_model=RAT_D_MODEL, n_blocks=RAT_NUM_BLOCKS,
    n_heads=RAT_NUM_HEADS, d_ff=128, dropout=RAT_DROPOUT, num_dim=X_num_tr.shape[1]
).to(device)


if device.type == "cuda":
    model = model.half()
    torch.backends.cuda.matmul.allow_tf32 = True

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

num_epochs = 3

target_effective_bs = 8192
accum_steps = max(1, target_effective_bs // bs_train)

total_steps = (num_epochs * len(tr_loader) + accum_steps - 1) // accum_steps
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=1e-3, total_steps=total_steps,
    pct_start=0.1, div_factor=10, final_div_factor=100
)

scaler = amp.GradScaler('cuda', enabled=(device.type == "cuda"))

In [6]:
# =========================
# Обучение RAT 
# =========================

model = model.to(device).float()

_non_fp32 = []
for n, p in model.named_parameters():
    if p.is_floating_point() and p.dtype != torch.float32:
        _non_fp32.append((f"param:{n}", str(p.dtype)))
for n, b in model.named_buffers():
    if b.is_floating_point() and b.dtype != torch.float32:
        _non_fp32.append((f"buffer:{n}", str(b.dtype)))
if _non_fp32:
    print("WARN: upcasting non-FP32 tensors:", _non_fp32)

try:
    scaler
except NameError:
    scaler = amp.GradScaler(enabled=(device.type == "cuda"))

steps_per_epoch_eff = math.ceil(len(tr_loader) / max(1, accum_steps))
try:
    del scheduler
except NameError:
    pass
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=1e-3,
    epochs=num_epochs,
    steps_per_epoch=steps_per_epoch_eff,
    pct_start=0.1,
    div_factor=10.0,
    final_div_factor=100.0,
)

for ep in range(1, num_epochs + 1):
    model.train()
    running = 0.0
    step_accum = 0
    opt_steps_done = 0

    for it, (cats_all, labs_all, nums, tgt) in enumerate(
        tqdm(tr_loader, desc=f"Epoch {ep}/{num_epochs} [train]"), start=1
    ):
        cats_all = cats_all.to(device, non_blocking=True)
        labs_all = labs_all.to(device, non_blocking=True)
        nums     = nums.to(device, non_blocking=True)
        tgt      = tgt.to(device, non_blocking=True)

        with amp.autocast('cuda', enabled=(device.type == "cuda")):
            logit = model(cats_all, labs_all, nums)
            loss = criterion(logit.float(), tgt) / accum_steps

        scaler.scale(loss).backward()
        step_accum += 1

        if step_accum % accum_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=3.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)

            scheduler.step() 
            opt_steps_done += 1
            step_accum = 0

        running += float(loss.detach().cpu())

        # ранняя очистка временных тензоров
        del cats_all, labs_all, nums, tgt, logit, loss
        if device.type == "cuda":
            torch.cuda.empty_cache()

    if step_accum > 0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=3.0)
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad(set_to_none=True)

        scheduler.step()
        opt_steps_done += 1

    print(f"[RAT] Epoch {ep} train loss: {running / max(1, len(tr_loader)):.4f}")

    model.eval()
    try:
        n_va = len(va_loader.dataset)
    except Exception:
        n_va = int(use_mask_va.sum())
    va_buf = np.empty(n_va, dtype=np.float32)
    write_ptr = 0

    with torch.no_grad(), amp.autocast('cuda', enabled=(device.type == "cuda")):
        for cats_all, labs_all, nums, tgt in tqdm(va_loader, desc=f"Epoch {ep}/{num_epochs} [valid]"):
            cats_all = cats_all.to(device, non_blocking=True)
            labs_all = labs_all.to(device, non_blocking=True)
            nums     = nums.to(device, non_blocking=True)

            logit = model(cats_all, labs_all, nums)
            prob = torch.sigmoid(logit.float()).squeeze(1)
            prob = torch.nan_to_num(prob, nan=0.5, posinf=1.0, neginf=0.0)

            b = prob.shape[0]
            va_buf[write_ptr:write_ptr + b] = prob.detach().cpu().numpy().astype(np.float32, copy=False)
            write_ptr += b

            del cats_all, labs_all, nums, logit, prob, tgt
            if device.type == "cuda":
                torch.cuda.empty_cache()

    va_auc = roc_auc_score(y_va[:write_ptr], va_buf[:write_ptr])
    print(f"[RAT] Epoch {ep}  Val AUC: {va_auc:.4f}")

    del va_buf
    gc.collect()
    if device.type == "cuda":
        torch.cuda.empty_cache()


Epoch 1/3 [train]:   0%|          | 1/1221 [00:01<29:25,  1.45s/it]/home/misha/lgbm_cuda/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
Epoch 1/3 [train]: 100%|██████████| 1221/1221 [06:20<00:00,  3.21it/s]


[RAT] Epoch 1 train loss: 0.2582


Epoch 1/3 [valid]: 100%|██████████| 258/258 [02:41<00:00,  1.59it/s]


[RAT] Epoch 1  Val AUC: 0.7237


Epoch 2/3 [train]: 100%|██████████| 1221/1221 [07:20<00:00,  2.77it/s]


...

[RAT] Epoch 2  Val AUC: 0.7272


Epoch 3/3 [train]: 100%|██████████| 1221/1221 [04:30<00:00,  4.52it/s]


[RAT] Epoch 3 train loss: 0.2434


Epoch 3/3 [valid]: 100%|██████████| 258/258 [02:23<00:00,  1.79it/s]


[RAT] Epoch 3  Val AUC: 0.7280


In [9]:
# =========================
# Экстракция вероятностей RAT для LGBM init_score (OOM-safe)
# =========================
def extract_probs_rat(ds: CTRDatasetRAT, bs=8192):
    pin = (device.type == "cuda")
    model.eval()

    while True:
        try:
            dl = DataLoader(
                ds,
                batch_size=bs,
                shuffle=False,
                num_workers=0,              
                pin_memory=pin,
                persistent_workers=False,
                collate_fn=rat_collate_infer,
                prefetch_factor=None,        
            )

            n = len(ds)
            out = np.empty(n, dtype=np.float32)
            idx = 0

            with torch.no_grad():
                for cats_all, labs_all, nums in tqdm(dl, desc=f"Extract RAT probs (bs={bs})", leave=False, disable=True):
                    cats_all = cats_all.to(device, non_blocking=True)
                    labs_all = labs_all.to(device, non_blocking=True)
                    nums     = nums.to(device, non_blocking=True)

                    with amp.autocast('cuda', enabled=(device.type == "cuda")):
                        logit = model(cats_all, labs_all, nums)
                    prob = torch.sigmoid(logit.float()).squeeze(1)
                    prob = torch.nan_to_num(prob, nan=0.5, posinf=1.0, neginf=0.0)

                    bsz = prob.shape[0]
                    out[idx:idx+bsz] = prob.detach().cpu().numpy().astype(np.float32, copy=False)
                    idx += bsz

                    # ранняя очистка
                    del cats_all, labs_all, nums, logit, prob
                    if device.type == "cuda":
                        torch.cuda.empty_cache()

            if device.type == "cuda":
                torch.cuda.synchronize()
            del dl
            gc.collect()
            return out[:idx]

        except RuntimeError as e:
            msg = str(e).lower()
            # ловим CUDA OOM/инициализацию/abort и уменьшаем батч
            if ("cuda" in msg) or ("out of memory" in msg) or ("initialization error" in msg) or ("device-side assert" in msg):
                if bs <= 1024:
                    raise
                if device.type == "cuda":
                    torch.cuda.empty_cache()
                bs = max(1024, bs // 2)
                print(f"[extract_probs_rat] CUDA issue: уменьшаю batch size до {bs} и повторяю…")
                continue
            else:
                raise


# корректные датасеты с РЕМЭПНУТЫМИ индексами (mask-индексы)
tr_infer_ds = CTRDatasetRAT(emb_tr, emb_tr, X_num_tr, None, retr_tr_idx_mask, y_tr)
va_infer_ds = CTRDatasetRAT(emb_va, emb_tr, X_num_va, None, retr_va_idx_mask, y_tr)

# умеренный стартовый bs; функция сама уменьшит при необходимости
tr_probs = extract_probs_rat(tr_infer_ds, bs=8192)
va_probs = extract_probs_rat(va_infer_ds, bs=8192)

rat_val_auc = roc_auc_score(y_va, va_probs.astype(np.float32, copy=False))
print(f"[RAT] Final Val AUC = {rat_val_auc:.4f}")

# подчистим всё, что могло держать CUDA/память, перед LGBM
del tr_infer_ds, va_infer_ds
if 'tr_loader' in globals(): del tr_loader
if 'va_loader' in globals(): del va_loader
if 'tr_dataset' in globals(): del tr_dataset
if 'va_dataset' in globals(): del va_dataset
gc.collect()
if device.type == "cuda":
    torch.cuda.synchronize()
    torch.cuda.empty_cache()

# =========================
# LightGBM поверх RAT (init_score) + engineered фичи
# =========================
lgb_train_df = pd.DataFrame(index=train_df.index)
lgb_valid_df = pd.DataFrame(index=valid_df.index)
for c in base_cat_cols:
    lgb_train_df[c] = train_df[c].astype("int32")
    lgb_valid_df[c] = valid_df[c].astype("int32")
cat_indices = [lgb_train_df.columns.get_loc(c) for c in base_cat_cols]
for c in num_cols:
    lgb_train_df[c] = train_df[c].astype("float32")
    lgb_valid_df[c] = valid_df[c].astype("float32")

def probs_to_logit(p):
    p = np.asarray(p, dtype=np.float32)
    p = np.clip(p, 1e-6, 1-1e-6)
    return np.log(p/(1-p)).astype(np.float32, copy=False)

tr_logit = probs_to_logit(tr_probs)
va_logit = probs_to_logit(va_probs)

dtrain = lgb.Dataset(lgb_train_df, label=y_tr, categorical_feature=cat_indices, free_raw_data=True)
dvalid = lgb.Dataset(lgb_valid_df, label=y_va, categorical_feature=cat_indices, reference=dtrain, free_raw_data=True)
dtrain.set_init_score(tr_logit)
dvalid.set_init_score(va_logit)


[RAT] Final Val AUC = 0.7280


20 min

In [ ]:
# LightGBM поверх RAT (init_score) + engineered фичи — версия с экономией RAM и безопасными удалениями
params = dict(
    objective="binary",
    metric="auc",
    learning_rate=0.05,
    num_leaves=256,
    max_bin=255,
    bin_construct_sample_cnt=50_000,  
    histogram_pool_size=256,       
    device_type="cuda",
    gpu_device_id=0,
    gpu_use_dp=False,             
    verbosity=-1,
)

print("[LGBM] Training on GPU with init_score ...")
callbacks = [
    lgb.early_stopping(stopping_rounds=150, first_metric_only=True),
    lgb.log_evaluation(period=200),
]

# init_score и немедленно освобождаем тяжёлые копии
dtrain.set_init_score(tr_logit)
dvalid.set_init_score(va_logit)
for _name in ("lgb_train_df", "lgb_valid_df"):
    if _name in globals():
        del globals()[_name]
gc.collect()

booster = lgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    valid_sets=[dvalid],
    valid_names=["valid"],
    callbacks=callbacks,
    keep_training_booster=False,
)

[LGBM] Training on GPU with init_score ...
Training until validation scores don't improve for 150 rounds
[200]	valid's auc: 0.728693
Early stopping, best iteration is:
[160]	valid's auc: 0.729797
Evaluated only: auc


In [13]:
feature_cols = list(base_cat_cols) + list(num_cols)

def predict_raw_score_in_chunks(booster, df, cols, num_iteration=None, chunk_rows=200_000):
    n = df.shape[0]
    out = np.empty(n, dtype=np.float32)
    start = 0
    while start < n:
        end = min(n, start + chunk_rows)
        X_chunk = df.iloc[start:end][cols]
        raw = booster.predict(X_chunk, raw_score=True, num_iteration=num_iteration)
        out[start:end] = np.asarray(raw, dtype=np.float32, order="C")
        start = end
    return out

tree_raw = predict_raw_score_in_chunks(
    booster, valid_df, feature_cols, num_iteration=booster.best_iteration, chunk_rows=200_000
)

tree_raw += va_logit
expit(tree_raw, out=tree_raw)
final_prob = tree_raw

print(f"[LGBM over RAT] Val AUC = {roc_auc_score(y_va, final_prob):.4f}")

del dtrain, dvalid, tree_raw, final_prob
gc.collect()
if device.type == "cuda":
    torch.cuda.synchronize()
    torch.cuda.empty_cache()

print(f"Готово за {time.perf_counter()-t0:.1f} c")


[LGBM over RAT] Val AUC = 0.7298
Готово за 8510.1 c
